In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

25/03/17 12:24:26 WARN Utils: Your hostname, Tees-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.91.185.118 instead (on interface en0)
25/03/17 12:24:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/17 12:24:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.version

'3.5.5'

In [3]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-17 12:24:38--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.46, 18.155.128.187, 18.155.128.222, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  3.44MB/s    in 25s     

2025-03-17 12:25:05 (2.48 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



# Repartition

In [5]:
!ls

homework.ipynb                  yellow_tripdata_2024-10.parquet


In [6]:
df = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [7]:
df = df.repartition(4)

In [8]:
df.write.parquet('yellow/2024/01')

In [9]:
!ls -al yellow/2024/01


total 190472
drwxr-xr-x  12 teeyeejian  staff       384 Mar 17 12:29 .
drwxr-xr-x   4 teeyeejian  staff       128 Mar 17 12:29 ..
-rw-r--r--   1 teeyeejian  staff         8 Mar 17 12:29 ._SUCCESS.crc
-rw-r--r--   1 teeyeejian  staff    183424 Mar 17 12:29 .part-00000-aa6baeaf-b560-4265-8cdb-0d4cb34b9c95-c000.snappy.parquet.crc
-rw-r--r--   1 teeyeejian  staff    183488 Mar 17 12:29 .part-00001-aa6baeaf-b560-4265-8cdb-0d4cb34b9c95-c000.snappy.parquet.crc
-rw-r--r--   1 teeyeejian  staff    183448 Mar 17 12:29 .part-00002-aa6baeaf-b560-4265-8cdb-0d4cb34b9c95-c000.snappy.parquet.crc
-rw-r--r--   1 teeyeejian  staff    183624 Mar 17 12:29 .part-00003-aa6baeaf-b560-4265-8cdb-0d4cb34b9c95-c000.snappy.parquet.crc
-rw-r--r--   1 teeyeejian  staff         0 Mar 17 12:29 _SUCCESS
-rw-r--r--   1 teeyeejian  staff  23477172 Mar 17 12:29 part-00000-aa6baeaf-b560-4265-8cdb-0d4cb34b9c95-c000.snappy.parquet
-rw-r--r--   1 teeyeejian  staff  23485343 Mar 17 12:29 part-00001-aa6baeaf-b560-4265-8cdb-0d4c

In [10]:
df.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', LongType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('Airport_fee', DoubleType(), True)])

In [11]:
df.registerTempTable('yellow_taxi')

/opt/homebrew/Cellar/apache-spark/3.5.5/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [13]:
spark.sql("""
SELECT
    date(tpep_pickup_datetime),
    count(*)
FROM
    yellow_taxi
GROUP BY 
    date(tpep_pickup_datetime)
""").show()

+--------------------+--------+
|tpep_pickup_datetime|count(1)|
+--------------------+--------+
|          2024-10-24|  137337|
|          2024-10-02|  113906|
|          2024-10-25|  136066|
|          2024-09-30|      12|
|          2024-10-22|  121106|
|          2024-10-18|  133213|
|          2024-10-08|  121402|
|          2024-10-10|  143206|
|          2024-10-20|  117129|
|          2024-10-01|  119118|
|          2024-10-04|  112431|
|          2024-10-15|  128893|
|          2024-10-28|  110595|
|          2024-10-29|  126931|
|          2024-10-17|  136330|
|          2024-10-31|  129394|
|          2024-10-07|  102014|
|          2024-11-01|      26|
|          2024-10-11|  128821|
|          2024-10-16|  134891|
+--------------------+--------+
only showing top 20 rows



In [19]:
spark.sql("""
SELECT
    tpep_pickup_datetime,
    tpep_dropoff_datetime,
    datediff(HOUR, tpep_pickup_datetime, tpep_dropoff_datetime) as trip_duration
FROM
    yellow_taxi
order by 
    trip_duration desc
""").show()

+--------------------+---------------------+-------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|trip_duration|
+--------------------+---------------------+-------------+
| 2024-10-16 13:03:49|  2024-10-23 07:40:53|          162|
| 2024-10-03 18:47:25|  2024-10-09 18:06:55|          143|
| 2024-10-22 16:00:55|  2024-10-28 09:46:33|          137|
| 2024-10-18 09:53:32|  2024-10-23 04:43:37|          114|
| 2024-10-20 13:30:52|  2024-10-24 06:57:38|           89|
| 2024-10-21 00:36:24|  2024-10-24 18:30:18|           89|
| 2024-10-22 16:04:52|  2024-10-25 14:22:49|           70|
| 2024-10-12 19:32:51|  2024-10-15 15:07:15|           67|
| 2024-10-17 17:58:18|  2024-10-20 12:02:18|           66|
| 2024-10-21 14:28:21|  2024-10-23 12:53:42|           46|
| 2024-10-20 18:58:28|  2024-10-22 13:17:00|           42|
| 2024-10-24 20:29:58|  2024-10-26 10:58:25|           38|
| 2024-10-23 23:52:02|  2024-10-25 09:49:06|           33|
| 2024-10-02 10:36:50|  2024-10-03 12:54:45|           2

In [20]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-17 12:53:54--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.6, 18.155.128.187, 18.155.128.46, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.01s   

2025-03-17 12:53:55 (924 KB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [23]:
taxi_zone_lookup = spark.read.csv('taxi_zone_lookup.csv', header=True)

In [24]:
taxi_zone_lookup.schema

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])

In [31]:
yello_taxi_joined = df.join(taxi_zone_lookup, df.PULocationID == taxi_zone_lookup.LocationID)

In [33]:
yello_taxi_joined.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+---------+--------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|LocationID|  Borough|                Zone|service_zone|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+---------+--------------------+------------+
|       1| 2024-10-07 

In [34]:
yello_taxi_joined.registerTempTable('yellow_taxi_joined')

/opt/homebrew/Cellar/apache-spark/3.5.5/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [35]:
spark.sql("""
SELECT 
    Zone,
    count(*)
FROM 
    yellow_taxi_joined
GROUP BY
    Zone
ORDER BY
    2
""").show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|Governor's Island...|       1|
|       Rikers Island|       2|
|       Arden Heights|       2|
|         Jamaica Bay|       3|
| Green-Wood Cemetery|       3|
|   Rossville/Woodrow|       4|
|       West Brighton|       4|
|       Port Richmond|       4|
|Eltingville/Annad...|       4|
|Charleston/Totten...|       4|
|         Great Kills|       6|
|        Crotona Park|       6|
|     Mariners Harbor|       7|
|Heartland Village...|       7|
|Saint George/New ...|       9|
|             Oakwood|       9|
|New Dorp/Midland ...|      10|
|       Broad Channel|      10|
|         Westerleigh|      12|
|     Pelham Bay Park|      12|
+--------------------+--------+
only showing top 20 rows

